In [10]:
import h5py
import random


In [2]:

def make_small_h5(input_path, output_path, sample_size=500, group_name="dataset2", seed=42):
    random.seed(seed)

    with h5py.File(input_path, "r") as fin:
        with h5py.File(output_path, "w") as fout:
            # Crear estructura
            fin_group = fin[group_name]
            fout_group = fout.create_group(group_name)
            fout_k = fout_group.create_group("keypoints")
            fout_e = fout_group.create_group("embeddings")
            fout_l = fout_group.create_group("labels") if "labels" in fin_group else None

            # Obtener clips válidos
            clips = list(fin_group["keypoints"].keys())
            random.shuffle(clips)
            selected = clips[:sample_size]

            for clip in selected:
                # Copiar keypoints
                data = fin_group["keypoints"][clip][:]
                fout_k.create_dataset(clip, data=data, compression="gzip")

                # Copiar embedding
                emb = fin_group["embeddings"][clip][:]
                fout_e.create_dataset(clip, data=emb)

                # Copiar label si hay
                if fout_l:
                    label = fin_group["labels"][clip][:]
                    fout_l.create_dataset(clip, data=label)

    print(f"✅ Guardado {sample_size} clips en '{output_path}'")

# Ejemplo de uso:
# make_small_h5("dataset_grande.h5", "dataset_pequeño.h5", sample_size=300)


In [9]:
def make_small_h5_ratio(input_path, output_path, sample_ratio=0.05, group_name="dataset2", seed=42):
    """
    Crea un .hdf5 reducido tomando una fracción aleatoria de los clips.
    - sample_ratio: proporción de clips a conservar (ej: 0.05 para 5%)
    """
    import h5py, random

    random.seed(seed)

    with h5py.File(input_path, "r") as fin:
        with h5py.File(output_path, "w") as fout:
            fin_group = fin[group_name]
            fout_group = fout.create_group(group_name)
            fout_k = fout_group.create_group("keypoints")
            fout_e = fout_group.create_group("embeddings")
            fout_l = fout_group.create_group("labels") if "labels" in fin_group else None

            # Lista de clips
            clips = list(fin_group["keypoints"].keys())
            total = len(clips)
            n = max(1, int(total * sample_ratio))

            print(f"➡️  Tomando {n} de {total} clips ({sample_ratio*100:.1f}%)")

            selected = random.sample(clips, n)

            for clip in selected:
                fout_k.create_dataset(clip, data=fin_group["keypoints"][clip][:], compression="gzip")
                fout_e.create_dataset(clip, data=fin_group["embeddings"][clip][:])
                if fout_l:
                    fout_l.create_dataset(clip, data=fin_group["labels"][clip][:])

    print(f"✅ Guardado en '{output_path}'")


In [7]:

data_path = "../../../../data/dataset_clean_clean.hdf5"
sample_ratio = 25
output_path = f"../../../../data/dataset_small_clean{sample_ratio}.hdf5"
!dir ../../../../data

copyfiles.ipynb  dataset6-video.py	   dataset_small50.hdf5
dataset1	 dataset7		   DataverseDownload.ipynb
dataset1.hdf5	 dataset_clean_clean.hdf5  note.md
dataset2	 dataset_clean.hdf5	   script.sh
dataset3	 dataset.hdf5		   srt_videos.ipynb
dataset4	 dataset_small1.hdf5	   transcripcion.ipynb
dataset5	 dataset_small25.hdf5
dataset6	 dataset_small2.hdf5


In [11]:


#make_small_h5(data_path, output_path, sample_size=sample_size, group_name="dataset2", seed=42)
make_small_h5_ratio(data_path, output_path, sample_ratio=sample_ratio/100, group_name="dataset2", seed=42)


➡️  Tomando 2114 de 8459 clips (25.0%)
✅ Guardado en '../../../../data/dataset_small_clean25.hdf5'


# clean dataset 

In [ ]:
def fix_keypoinys(keypoints):
    T, N, _ = keypoints.shape
    filtered = keypoints[:, 117:, :].clone()  # Clonar para evitar modificar el original
    return filtered


In [ ]:
from torch import as_tensor
import torch

def clean_fn(keypoints_np):
    keypoints = as_tensor(keypoints_np, dtype=torch.float32)
    cleaned = fix_keypoinys(keypoints)
    return cleaned.numpy()


In [ ]:
def make_clean_h5_all_groups(input_path, output_path, clean_keypoints_fn=None):
    """
    Crea un nuevo archivo HDF5 copiando todos los grupos y clips del original,
    aplicando una función de limpieza a los keypoints.

    - clean_keypoints_fn: función que recibe un array (T, J, 2) y retorna el keypoint limpio.
    """
    import h5py

    with h5py.File(input_path, "r") as fin:
        with h5py.File(output_path, "w") as fout:
            group_names = list(fin.keys())
            print(f"📁 Grupos encontrados: {group_names}")

            for group_name in group_names:
                print(f"\n➡️ Procesando grupo '{group_name}'...")
                fin_group = fin[group_name]
                fout_group = fout.create_group(group_name)

                fout_k = fout_group.create_group("keypoints")
                fout_e = fout_group.create_group("embeddings")
                fout_l = fout_group.create_group("labels") if "labels" in fin_group else None

                clips = list(fin_group["keypoints"].keys())
                print(f"  🧩 {len(clips)} clips")

                for clip in clips:
                    keypoints = fin_group["keypoints"][clip][:]
                    if clean_keypoints_fn:
                        keypoints = clean_keypoints_fn(keypoints)

                    fout_k.create_dataset(clip, data=keypoints, compression="gzip")
                    fout_e.create_dataset(clip, data=fin_group["embeddings"][clip][:])
                    if fout_l:
                        fout_l.create_dataset(clip, data=fin_group["labels"][clip][:])

    print(f"\n✅ Dataset limpio guardado en: {output_path}")


In [ ]:

make_clean_h5_all_groups(
    input_path="../../../../data/dataset1.hdf5",
    output_path="../../../../data/dataset_clean.hdf5",
    clean_keypoints_fn=clean_fn
)


In [ ]:
import pandas as pd
import numpy as np
import h5py

def extract_kp0_variance(input_path):
    """
    Recorre un HDF5 con estructura (grupo -> clips -> keypoints) y 
    calcula la varianza temporal del keypoint 0 por clip.
    """
    registros = []

    with h5py.File(input_path, "r") as fin:
        for group_name in fin.keys():
            group = fin[group_name]
            for clip_name in group["keypoints"].keys():
                keypoints = group["keypoints"][clip_name][:]  # shape: (T, J, 2)

                if keypoints.shape[1] <= 0:
                    continue  # skip si no hay joints

                kp0 = keypoints[:, 0, :]  # (T, 2)
                var_x = np.var(kp0[:, 0])
                var_y = np.var(kp0[:, 1])
                var_total = var_x + var_y

                registros.append({
                    "grupo": group_name,
                    "clip": clip_name,
                    "var_x": var_x,
                    "var_y": var_y,
                    "var_total": var_total
                })

    df = pd.DataFrame(registros)
    return df



df = extract_kp0_variance("../../../../data/dataset_clean.hdf5")

In [ ]:
import matplotlib.pyplot as plt

# Percentil 95 para decidir max_seq_len sin truncar tanto
p95 = int(np.percentile(df['var_x'], 80))

# Visualización
plt.hist(df['var_x'], bins=30, color='skyblue', edgecolor='black')
plt.axvline(p95, color='red', linestyle='--', label=f'95% ≤ {p95}')
plt.title("Distribución de longitudes de secuencia")
plt.xlabel("Frames por muestra")
plt.ylabel("Frecuencia")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
def detect_side_dominance(keypoints, threshold=0.6):
    """
    keypoints: (T, J, 2)
    Retorna "left" o "right" según dónde pasó más tiempo el keypoint 0.
    """
    kp0_x = keypoints[:, 0, 0]  # eje x del keypoint 0
    mid_x = np.median(kp0_x)  # usar mediana como frontera

    left_count = np.sum(kp0_x < mid_x)
    right_count = np.sum(kp0_x >= mid_x)
    total = len(kp0_x)

    if left_count / total >= threshold:
        return "left"
    elif right_count / total >= threshold:
        return "right"
    else:
        return "mixed"

def clean_by_dominant_side(keypoints, threshold=0.6):
    """
    Elimina frames donde el keypoint 0 cambia de lado respecto al dominante.
    """
    kp0_x = keypoints[:, 0, 0]
    mid_x = np.median(kp0_x)
    dominant_side = detect_side_dominance(keypoints, threshold)

    if dominant_side == "left":
        mask = kp0_x < mid_x
    elif dominant_side == "right":
        mask = kp0_x >= mid_x
    else:
        return keypoints  # no filtramos si es mixto

    return keypoints[mask]


In [ ]:
def plot_keypoints_comparison(original, filtrado, joint_id=0):
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    fig.suptitle(f"Comparación del keypoint {joint_id} (original vs filtrado)", fontsize=14)

    ax[0].plot(original[:, joint_id, 0], original[:, joint_id, 1], 'bo-', alpha=0.5)
    ax[0].set_title(f"Original - {original.shape[0]} frames")
    ax[0].invert_yaxis()
    ax[0].grid(True)
    ax[0].set_aspect("equal")

    ax[1].plot(filtrado[:, joint_id, 0], filtrado[:, joint_id, 1], 'ro-', alpha=0.5)
    ax[1].set_title(f"Filtrado - {filtrado.shape[0]} frames")
    ax[1].invert_yaxis()
    ax[1].grid(True)
    ax[1].set_aspect("equal")

    plt.tight_layout()
    plt.show()


In [ ]:
path = "../../../../data/dataset_clean.hdf5"

import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import groupby
from operator import itemgetter

# ------------ Filtros ------------

def detect_side_dominance(keypoints, threshold=0.6):
    kp0_x = keypoints[:, 0, 0]
    mid_x = np.median(kp0_x)
    left = np.sum(kp0_x < mid_x)
    right = np.sum(kp0_x >= mid_x)
    return "left" if left/len(kp0_x) >= threshold else "right" if right/len(kp0_x) >= threshold else "mixed"

def clean_by_dominant_side(keypoints, threshold=0.6):
    kp0_x = keypoints[:, 0, 0]
    mid_x = np.median(kp0_x)
    side = detect_side_dominance(keypoints, threshold)
    if side == "left":
        return keypoints[kp0_x < mid_x]
    elif side == "right":
        return keypoints[kp0_x >= mid_x]
    return keypoints

def extract_stable_segment(keypoints, margin=15):
    T = keypoints.shape[0]
    mask = np.zeros(T, dtype=bool)
    for t in range(1, T):
        x1 = keypoints[t-1, 1, 0]
        x4 = keypoints[t-1, 4, 0]
        x_min = min(x1, x4) - margin
        x_max = max(x1, x4) + margin
        head_x = keypoints[t, 0, 0]
        if x_min <= head_x <= x_max:
            mask[t] = True

    indices = np.where(mask)[0]
    groups = [list(map(itemgetter(1), g)) for k, g in groupby(enumerate(indices), lambda i: i[0]-i[1])]
    return keypoints[max(groups, key=len)] if groups else None

def clean_keypoints_smart(keypoints, min_len=30):
    stable = extract_stable_segment(keypoints)
    if stable is not None and len(stable) >= min_len:
        return stable, "stable"
    return clean_by_dominant_side(keypoints), "dominant"

# ------------ Visualización ------------

def visualize_first_clip(file_path):
    with h5py.File(file_path, "r") as f:
        group = list(f.keys())[0]
        clip = list(f[group]["keypoints"].keys())[0]
        keypoints = f[group]["keypoints"][clip][:]

    filtered, method = clean_keypoints_smart(keypoints)

    df = pd.DataFrame({
        "frame": np.arange(len(keypoints)),
        "kp0_x_original": keypoints[:, 0, 0],
        "kp0_x_filtered": np.nan
    })

    filtered_x = set(filtered[:, 0, 0])
    mask = np.isin(keypoints[:, 0, 0], list(filtered_x))
    df.loc[mask, "kp0_x_filtered"] = keypoints[mask, 0, 0]

    # Graficar
    plt.figure(figsize=(12, 4))
    plt.plot(df["frame"], df["kp0_x_original"], label="Original", alpha=0.5)
    plt.plot(df["frame"], df["kp0_x_filtered"], label="Filtrado", alpha=0.8)
    plt.title(f"Keypoint 0 - Eje X | Método: {method}")
    plt.xlabel("Frame")
    plt.ylabel("Posición X")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return df

# ------------ Uso ------------

# Reemplaza esto por tu archivo
df_cmp = visualize_first_clip(path)


Ahora con esto podemos ver si hay un cambio abrupto y determinar que el open pose cambio de foco

In [ ]:
df_cmp

In [ ]:
from itertools import groupby
from operator import itemgetter

# ---------- Interpolador ----------

def interpolate_unstable_frames(keypoints, margin=15):
    keypoints_interp = keypoints.copy()
    T = keypoints.shape[0]
    valid_mask = np.zeros(T, dtype=bool)

    for t in range(1, T):
        x1 = keypoints[t-1, 1, 0]
        x4 = keypoints[t-1, 4, 0]
        x_min = min(x1, x4) - margin
        x_max = max(x1, x4) + margin
        head_x = keypoints[t, 0, 0]
        if x_min <= head_x <= x_max:
            valid_mask[t] = True
    valid_mask[0] = True

    invalid_idx = np.where(~valid_mask)[0]
    valid_idx = np.where(valid_mask)[0]

    for i in invalid_idx:
        before = valid_idx[valid_idx < i]
        after = valid_idx[valid_idx > i]
        if len(before) == 0 or len(after) == 0:
            continue
        t0 = before[-1]
        t1 = after[0]
        alpha = (i - t0) / (t1 - t0)
        keypoints_interp[i] = (1 - alpha) * keypoints[t0] + alpha * keypoints[t1]

    return keypoints_interp

# ---------- Adaptación de limpieza + tracking varianza ----------

def make_clean_h5_all_groups_with_stats(input_path, output_path, clean_keypoints_fn=None):
    df_before = []
    df_after = []

    with h5py.File(input_path, "r") as fin:
        with h5py.File(output_path, "w") as fout:
            for group_name in fin.keys():
                fin_group = fin[group_name]
                fout_group = fout.create_group(group_name)

                fout_k = fout_group.create_group("keypoints")
                fout_e = fout_group.create_group("embeddings")
                fout_l = fout_group.create_group("labels") if "labels" in fin_group else None

                for clip in fin_group["keypoints"].keys():
                    keypoints = fin_group["keypoints"][clip][:]
                    var_before = np.var(keypoints[:, 0, :], axis=0).sum()
                    df_before.append({
                        "group": group_name,
                        "clip": clip,
                        "var_x": np.var(keypoints[:, 0, 0]),
                        "var_y": np.var(keypoints[:, 0, 1]),
                        "var_total": var_before
                    })

                    if clean_keypoints_fn:
                        keypoints = clean_keypoints_fn(keypoints)

                    var_after = np.var(keypoints[:, 0, :], axis=0).sum()
                    df_after.append({
                        "group": group_name,
                        "clip": clip,
                        "var_x": np.var(keypoints[:, 0, 0]),
                        "var_y": np.var(keypoints[:, 0, 1]),
                        "var_total": var_after
                    })

                    fout_k.create_dataset(clip, data=keypoints, compression="gzip")
                    fout_e.create_dataset(clip, data=fin_group["embeddings"][clip][:])
                    if fout_l:
                        fout_l.create_dataset(clip, data=fin_group["labels"][clip][:])

    return pd.DataFrame(df_before), pd.DataFrame(df_after)

# Simularemos ruta de uso
i_path = "../../../../data/dataset_clean.hdf5"
o_path = "../../../../data/dataset_clean_clean.hdf5"

df_before, df_after = make_clean_h5_all_groups_with_stats(i_path, o_path, interpolate_unstable_frames)


In [ ]:
df_diff = df_before.copy()
df_diff["var_total_after"] = df_after["var_total"]
df_diff["delta_var"] = df_diff["var_total"] - df_diff["var_total_after"]
df_diff["rel_reduction_%"] = 100 * df_diff["delta_var"] / df_diff["var_total"]


In [ ]:
print("Antes (total):", df_before["var_total"].sum())
print("Después (total):", df_after["var_total"].sum())
print("Reducción total (%):", 100 * (df_before["var_total"].sum() - df_after["var_total"].sum()) / df_before["var_total"].sum())


In [ ]:
# Mejores
print("🔽 Mayor reducción de varianza:")
df_diff.sort_values("delta_var", ascending=False)


In [ ]:


# Empeoramientos raros
print("🔼 Clips donde subió la varianza (deberían ser pocos o ninguno):")
df_diff[df_diff["delta_var"] < 0].sort_values("delta_var")


In [ ]:

plt.figure(figsize=(10,5))
plt.scatter(df_diff["var_total"], df_diff["var_total_after"], alpha=0.5)
plt.plot([df_diff["var_total"].min(), df_diff["var_total"].max()],
         [df_diff["var_total"].min(), df_diff["var_total"].max()],
         linestyle='--', color='gray')
plt.xlabel("Varianza antes")
plt.ylabel("Varianza después")
plt.title("Comparación de varianza total por clip (keypoint 0)")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
pivot = df_diff.pivot_table(index="group", values=["var_total", "var_total_after"], aggfunc="mean")
pivot["rel_reduction_%"] = 100 * (pivot["var_total"] - pivot["var_total_after"]) / pivot["var_total"]

plt.figure(figsize=(10,4))
sns.heatmap(pivot[["rel_reduction_%"]], annot=True, fmt=".1f", cmap="coolwarm")
plt.title("Reducción promedio de varianza por grupo (%)")
plt.tight_layout()
plt.show()


In [ ]:
print("🏆 Top clips con mayor reducción total:")
df_diff.sort_values("delta_var", ascending=False).head(10)
